<a href="https://colab.research.google.com/github/IgorAkO/Simulative_ML/blob/main/1_Basic%20Python/S_ML_1_10_1_Combined_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1 кейс

**Ваша задача написать функцию `process_files`, которая принимает на вход два пути к папкам. Из первой папки необходимо выбрать все "чеки" (файлы по шаблону из условия), а во вторую папку сохранить один объединенный чек (отсортированный по дате, а затем по продукту) под названием `combined_data.csv`.**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить папку с файлами. После выполнения, в папке `reports_main` будут храниться все присланные магазинами чеки.

In [1]:
!wget https://github.com/vs8th/reports/archive/main.zip

import zipfile

with zipfile.ZipFile("main.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

!rm main.zip

--2026-02-18 02:13:21--  https://github.com/vs8th/reports/archive/main.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Vs8th/reports/zip/refs/heads/main [following]
--2026-02-18 02:13:21--  https://codeload.github.com/Vs8th/reports/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ]   3.22K  --.-KB/s    in 0s      

2026-02-18 02:13:21 (54.1 MB/s) - ‘main.zip’ saved [3296]



Чтобы посмотреть как выглядят подходящие для объединения чеки выполните следующую ячейку.

In [ ]:
import pandas as pd

df = pd.read_csv('reports-main/2023-02-17-05-38-2.csv', sep=";")
df

,date,product,store,cost
0,2023-02-17,product_0,store_2,10
1,2023-02-17,product_1,store_2,20
2,2023-02-17,product_2,store_2,30
3,2023-02-17,product_3,store_2,40
4,2023-02-17,product_4,store_2,50


**Решение**

Напишите свое решение ниже

**Примечание**

Не все файлы подходящие по названию, будут подходить по содержанию. Там может оказаться лишний столбец, например. Ориентируйтесь на столбцы из чека выше - это то, что вас интересует. Остальные столбцы можно просто отбросить.

**Важно**: разделителем файла на выходе должна быть запятая.

In [9]:
from glob import glob
import os
import re
import csv

def process_files(src_folder, dest_folder):
  # I Создаем список подходящих файлов (file_list)
  # Все csv файлы в src_folder
  file_list = glob(src_folder + '/*.csv')
  file_list.sort()

  # Фильтруем список файлов по имени
  filt = r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d+.csv'
  filt_path = f'{src_folder}/{filt}'
  i = 0
  while i < len(file_list):
    if re.match(filt_path, file_list[i]) == None:
      file_list.pop(i)
    else:
      i += 1

  # II Создаем объединенный набор данных combined_data.csv (output_file)
  # Проверка наличия папки dest_folder
  if os.path.exists(dest_folder) == False:
    os.mkdir(dest_folder)

  # Пишем в combined_data.csv (output_file) первую строку с заголовками
  path_output_file = f'{dest_folder}/combined_data.csv'
  if os.path.exists(path_output_file) == False:
    with open(path_output_file, 'w', newline='') as output_file, \
    open(file_list[0], 'r', newline='') as input_file:
      writer = csv.writer(output_file, delimiter=',')
      reader = csv.reader(input_file, delimiter=';')
      str0_output = next(reader)     # строка с заголовками
      writer.writerow(str0_output)
  else:
    with open(path_output_file, 'r', newline='') as output_file:
      reader = csv.reader(output_file, delimiter=',')
      str0_output = next(reader)     # строка с заголовками output_file

  # Добавляем данные в output_file из file_list (input_file)
  with open(path_output_file, 'a', newline='') as output_file:
    writer = csv.writer(output_file, delimiter=',')

    for file_path in file_list:
      with open(file_path, 'r', newline='') as input_file:
        reader = csv.reader(input_file, delimiter=';')
        str0_input = next(reader) # Cтрока с заголовками input_file

        # Пишем строки после первой строки
        if str0_input == str0_output:
          for row in reader:
            writer.writerow(row)
        else:
          # Удаляем лишние элементы из строк
          for row in reader:
            dict_row = {col_name: val for col_name, val in zip(str0_input, row)}
            row = [dict_row[x] for x in str0_output if x in dict_row]
            writer.writerow(row)

src_folder = 'reports-main'
dest_folder = 'comb_reports'
process_files(src_folder, dest_folder)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [11]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv

import pandas as pd

user_answer = pd.read_csv(f'{dest_folder}/combined_data.csv')
correct_answer = pd.read_csv('data.csv')

--2026-02-18 02:31:28--  https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv [following]
--2026-02-18 02:31:29--  https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 984 [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]     984  --.-KB/s    in 0s      

2026-02-18 02:31:29 (40.5 MB/s) - ‘data.csv’ saved [984/984]



In [12]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!')

Поздравляем, Вы справились и успешно прошли все проверки!
